In [ ]:
import pandas as pd
import numpy as np

# Chargement des fichiers originaux
df_wax = pd.read_csv("/home/riccardo/Visual_Studio_Code/freelance/Synthetique_data/simulation_mesures_avec_nom.csv")
df_metal = pd.read_csv("/home/riccardo/Visual_Studio_Code/freelance/Synthetique_data/simulation_mesures_metal.csv")

# Mise à jour des noms de colonnes pour correspondre à ceux fournis par l'utilisateur
df_unique_cotes_wax = df_wax.drop_duplicates(subset="Nom_Cote", keep="first").copy()
df_unique_cotes_metal = df_metal.drop_duplicates(subset="Nom_Cote", keep="first").copy()


In [ ]:
def simuler_mesures_corrigé(
    df,
    nom_prefixe="Cire_",
    surcote_mm=0.15,
    resserrement_tol=0.5,
    bruit_mesure_mm=0.01
):
    import pandas as pd
    import numpy as np

    lignes = []

    # Conversion éventuelle de la colonne "Cote"
    if df["Cote"].dtype == 'object':
        df["Cote"] = df["Cote"].str.replace(",", ".").astype(float)

    for _, row in df.iterrows():
        nom_cote = row["Nom_Cote"]
        cote_plan = row["Cote"]  # valeur nominale métal
        valeur_nominale = row["Mesure_Theorique"]  # base pour surcote
        tol_plus = row["Tolérance_Max"] - cote_plan
        tol_moins = row["Tolérance_Min"] - cote_plan

        # Surcote appliquée
        nouvelle_nominale = valeur_nominale + surcote_mm

        # Tolérances resserrées autour de la valeur attendue
        nouvelle_tol_plus = (tol_plus * resserrement_tol) + cote_plan
        nouvelle_tol_moins = (tol_moins * resserrement_tol) + cote_plan

        # Écart-type basé sur tolérance
        sigma = min(nouvelle_tol_plus - cote_plan, abs(nouvelle_tol_moins - cote_plan)) / 3
        ecarts = np.random.normal(loc=0, scale=sigma, size=100)
        mesures_theoriques = np.round(nouvelle_nominale + ecarts, 4)

        # Bruit de mesure (si activé)
        bruit = np.random.normal(loc=0, scale=bruit_mesure_mm, size=100)
        valeurs_mesurees = np.round(mesures_theoriques + bruit, 4)

        for i, (theorique, mesuree) in enumerate(zip(mesures_theoriques, valeurs_mesurees)):
            lignes.append({
                "Date": pd.Timestamp.today().normalize(),
                "Serial": f"{nom_prefixe}{i+1:03}",
                "OF": "SIM",
                "Nom_Cote": nom_prefixe + nom_cote,
                "Mesure": mesuree,
                "Nominal": cote_plan,
                "Tolérance_Min": nouvelle_tol_moins,
                "Tolérance_Max": nouvelle_tol_plus
            })

    return pd.DataFrame(lignes)


In [15]:
# Relancer la simulation avec les bons noms de colonnes
df_simule_cire = simuler_mesures_corrigé(df_unique_cotes)

# Fusion avec les données d'origine
df_comparatif = pd.concat([df, df_simule_cire], ignore_index=True)

output_path_cire = "/home/riccardo/Visual_Studio_Code/freelance/Synthetique_data/simulation_mesures_cire.csv"
df_simule_cire.to_csv(output_path_cire, index=False)
print(f"✅ Fichier exporté : {output_path_cire}")

✅ Fichier exporté : /home/riccardo/Visual_Studio_Code/freelance/Synthetique_data/simulation_mesures_cire.csv
